In [0]:
cosmosendpoint = "<YourEndpoint>"
cosmosmasterkey = "<YourKey>"
cosmosserviceprincipalpassword=""
accountDataResolverName = "com.azure.cosmos.spark.samples.MasterKeyAccountDataResolver"
#accountDataResolverName = "com.azure.cosmos.spark.samples.ServicePrincipalAccountDataResolver"
#accountDataResolverName = "com.azure.cosmos.spark.samples.ManagedIdentityAccountDataResolver"

In [0]:
import base64
import os


cert_file= open("/Workspace/Users/someuser@abc.com/someCert.pem","rb")
cert_data_binary = cert_file.read()
cert_data = (base64.b64encode(cert_data_binary)).decode('ascii')


In [0]:
import random
import string
import uuid
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
 
def random_string_generator(str_size, allowed_chars):
    return ''.join(random.choice(allowed_chars) for x in range(str_size))
 
pk = str(uuid.uuid4())    
print("pk: " + pk)
    
chars = string.ascii_letters
data = [\
    (str(uuid.uuid4()), pk, "", None, 0, random_string_generator(16000, chars)),\
    (str(uuid.uuid4()), pk, "", None, 0, random_string_generator(random.randint(16000, 170000), chars)),\
  ]

schema = StructType([ \
    StructField("id",StringType(),False), \
    StructField("pk",StringType(),False), \
    StructField("emptycolumn",StringType(),True), \
    StructField("nullcolumn",StringType(),True), \
    StructField("defaultcolumn",IntegerType(),True), \
    StructField("largecolumn", StringType(), True)\
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=100)

writecfg = {
  "spark.cosmos.accountendpoint": cosmosendpoint,
  "spark.cosmos.accountDataResolverServiceName": accountDataResolverName,
  "spark.cosmos.database": "Test",
  "spark.cosmos.container": "TestItems",
  "spark.cosmos.write.strategy": "itemappend",
  "spark.cosmos.write.bulk.enabled": "true",  
  "cosmos.auth.sample.enabled": "true",
  # masterkey
  #"cosmos.auth.sample.authType": "masterkey",
  #"cosmos.auth.sample.key.secret": cosmosmasterkey,
  #
  # aad auth with managed identity
  #"cosmos.auth.sample.authType": "managedidentity",
  #"cosmos.auth.sample.tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionId": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop"
  #
  # aad auth with service principal (password)
  #"cosmos.auth.sample.authType": "serviceprincipal",
  #"cosmos.auth.sample.tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionId": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop",
  #"cosmos.auth.sample.serviceprincipal.clientId": "bd559cf4-786d-43ae-9ff6-eb83c5952c73",
  #"cosmos.auth.sample.serviceprincipal.clientsecret": cosmosserviceprincipalpassword
  #
  # aad auth with service principal (cert)
  #"cosmos.auth.sample.authType": "serviceprincipal",
  #"cosmos.auth.sample.tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  #"cosmos.auth.sample.subscriptionId": "8fba6d4f-7c37-4d13-9063-fd58ad2b86e2",
  #"cosmos.auth.sample.resourcegroupname": "fabianm-oltp-spark-workshop",
  #"cosmos.auth.sample.serviceprincipal.clientId": "88436299-945f-4824-8183-2cbddf981388",
  #"cosmos.auth.sample.serviceprincipal.cert": cert_data
}

df\
  .write \
  .format("cosmos.oltp") \
  .mode("append") \
  .options(**writecfg) \
  .save()
